In [3]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [4]:
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from glob import glob

In [5]:
IMAGE_SIZE=[224,224]

vgg16 = VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [6]:
## do not train the weights for the all the layers
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
folders = glob('Tomato-Disease/train/*')

In [8]:
folders

['Tomato-Disease/train\\Tomato___Bacterial_spot',
 'Tomato-Disease/train\\Tomato___Early_blight',
 'Tomato-Disease/train\\Tomato___healthy',
 'Tomato-Disease/train\\Tomato___Late_blight',
 'Tomato-Disease/train\\Tomato___Leaf_Mold',
 'Tomato-Disease/train\\Tomato___Septoria_leaf_spot',
 'Tomato-Disease/train\\Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato-Disease/train\\Tomato___Target_Spot',
 'Tomato-Disease/train\\Tomato___Tomato_mosaic_virus',
 'Tomato-Disease/train\\Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [9]:
#Adding the flatten layer
flatten = Flatten()(vgg16.output)

In [10]:
output = Dense(len(folders),activation='softmax')(flatten)

In [11]:
model = Model(inputs=vgg16.input,outputs=output)


In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
train_data = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_data = ImageDataGenerator(rescale = 1./255)

In [15]:
#training_set = train_data.flow_from_directory('Tomato-Disease/train',target_size=(244,24),batch_size=32,class_mode='categorical')
# Make sure you provide the same target size as initialied for the image size
training_set = train_data.flow_from_directory('Tomato-Disease/test',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2038 images belonging to 10 classes.


In [16]:
#testing_set = test_data.flow_from_directory('Tomato-Disease/test',target_size=(224,244),batch_size=32,class_mode='categorical')
testing_set = test_data.flow_from_directory('Tomato-Disease/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2038 images belonging to 10 classes.


In [17]:
len(training_set),len(testing_set)

(64, 64)

In [18]:
r = model.fit_generator(training_set,validation_data=testing_set,epochs=50,
                        steps_per_epoch=len(training_set),
                        validation_steps=len(testing_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
64/64 [==============================] - 670s 10s/step - loss: 1.5938 - accuracy: 0.5182 - val_loss: 0.8939 - val_accuracy: 0.6884
Epoch 2/50
64/64 [==============================] - 698s 11s/step - loss: 0.6874 - accuracy: 0.7733 - val_loss: 0.5899 - val_accuracy: 0.7915
Epoch 3/50
64/64 [==============================] - 710s 11s/step - loss: 0.4682 - accuracy: 0.8553 - val_loss: 0.3811 - val_accuracy: 0.8788
Epoch 4/50
64/64 [==============================] - 660s 10s/step - loss: 0.4106 - accuracy: 0.8675 - val_loss: 0.2769 - val_accuracy: 0.9112
Epoch 5/50
64/64 [==============================] - 1127s 18s/step - loss: 0.2957 - accuracy: 0.9127 - val_loss: 0.2245 - val_accuracy: 0.9362
Epoch 6/50
64/64 [==============================] - ETA: 0s - loss: 0.2588 - accuracy: 0.9161

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# Display Loss
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label='Val loss')
plt.legend()
plt.show()

#Display Accuracy
plt.plot(r.history['accuracy'], label='Accuracy')
plt.plot(r.history['val_accuracy'],label='Val Accuracy')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.models import load_model
md = model.save('vgg16.h5')

In [ ]:
y_pred = model.predict(testing_set)

In [ ]:
y_pred

In [ ]:
model = load_model('vgg16.h5')

In [ ]:
img = image.load_img('Tomato-Disease/valid/Tomato___Tomato_Yellow_Leaf_Curl_Virus/1.jpg',target_size=[224,224])
x = image.img_to_array(img)
x = x/255

In [ ]:
x = np.expand_dims(x,axis=0)
img_data = preprocess_input(x)
img_data.shape


In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)
a